# Subpart 7: Labs required

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math

In [2]:
%matplotlib inline
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()

Assumptions:
* No. of cases per state = Positives on 10.04.2020 from StatewiseTestingDetails.csv 
* State-wise Test positivity rate (TPR)% = mean(positive cases/totalsamples) in that State till 10.04.2020
* Projected postive cases computed using 10% daily hike (given)
* Total tests computed from state wise TPR %

Reading files 

In [3]:
labs = pd.read_csv('ICMRTestingLabs.csv',usecols=['lab','state'])
tests = pd.read_csv('StatewiseTestingDetails.csv', parse_dates =['Date'], dayfirst=True)


Filtering state-wise latest records and computing TPR

In [28]:
tests = tests[tests.Date<='2020-04-10']
tests['TPR']=tests.Positive/tests.TotalSamples
tests['Mean_TPR']=tests.groupby('State')['TPR'].transform('mean')

tests['max']=tests.groupby('State')["Date"].transform('max')
tests=tests.reset_index(drop=True)
rows = tests.shape[0]
i=0
A=[None]*rows
for i in range(rows):
    A[i]=[tests.at[i,'Date']==tests.at[i,'max']]
tests=tests[A]

tests=tests[['State','TotalSamples','Positive','Mean_TPR']].reset_index(drop=True)
tests= tests.rename({'TotalSamples': 'TotalSamples_10th', 'Positive': 'Positive_10th'}, axis=1)
TPR=tests[['State','Mean_TPR']]
TPR

AttributeError: 'DataFrame' object has no attribute 'Date'

* Computing projected posititves on 20.04.2020 [given 10% daily hike in positives]
* Computing total samples tests on 20.04.2020 from projected positives [using state-wise Test Positivity Rate(TPR);  (total samples x TPR) = positives]


* pos_19  = total positives till 19.04.2020
* pos_20  = total positives till 20.04.2020
* pos_new = positives on 20.04.2020
* total_new = total samples tests on 20.04.2020

In [20]:
tests['pos_19']=tests.Positive_10th
i=0
for i in range(9):
    tests['pos_19']=tests['pos_19']*1.1
    
tests['pos_20']=tests.pos_19*1.1
tests['pos_new']=tests.pos_20-tests.pos_19
tests['total_new']=tests.pos_new / tests.Mean_TPR
tests['total_new']=tests['total_new'].apply(np.ceil)
tests['pos_new']=tests['pos_new'].apply(np.ceil)
tests['req_labs']=tests.total_new/100
tests['req_labs']=tests['req_labs'].apply(np.ceil)
tests=tests[['State','pos_new','total_new','req_labs']]


In [21]:
labs = pd.read_csv('ICMRTestingLabs.csv',usecols=['lab','state'])
labs['No. of Labs']=labs.groupby('state')['state'].transform('count')
labs = labs.drop('lab',1)
labs.rename(columns = {'state':'State','No. of Labs':'labs_cur'}, inplace = True)
labs = labs.drop_duplicates().reset_index(drop=True)
labs=labs.merge(tests)
labs

,State,labs_cur,pos_new,total_new,req_labs
0,Arunachal Pradesh,1,1.0,49.0,1.0
1,Andhra Pradesh,7,87.0,1319.0,14.0
2,Assam,6,7.0,512.0,6.0
3,Bihar,6,15.0,1446.0,15.0
4,Chandigarh,3,5.0,41.0,1.0
5,Chhattisgarh,3,5.0,680.0,7.0
6,Delhi,18,213.0,2936.0,30.0
7,Gujarat,14,90.0,1917.0,20.0
8,Goa,1,2.0,79.0,1.0
9,Haryana,11,39.0,779.0,8.0


In [23]:
df=labs
df['additional_labs']=df.req_labs - df.labs_cur
df=df.sort_values(by ='additional_labs', ascending = 0).reset_index(drop=True) 
df=df[df['additional_labs']>0]
df=df[['State','additional_labs']]
df=df.pivot_table(index='State')
df.iplot(kind='bar')